## Установка и импорт зависимостей

In [1]:
!pip install pandas numpy tqdm CurrencyConverter

In [2]:
import pandas
import re
import numpy
from tqdm import tqdm
import helpers
from currency_converter import CurrencyConverter
cc = CurrencyConverter()

## Загрузка данных

In [3]:
df: pandas.DataFrame = pandas.read_csv('real_estate_data.csv', low_memory=False, index_col=0)

### Замена значений из таблицы для лучшего понимания

In [4]:
replace_map = {
}
df = df.replace(replace_map)

### Определяю столбцы без уникальных значений и убираю их

In [5]:
zero_unique_collumns = []
for name, col in df.iteritems():
    print(name + " : " + str(len(col.unique())))
    if(len(col.unique()) <= 1):
        zero_unique_collumns.append(name)

type : 1
sub_type : 12
start_date : 181
end_date : 182
listing_type : 3
tom : 181
building_age : 15
total_floor_count : 13
floor_no : 36
room_count : 37
size : 944
address : 7842
furnished : 1
heating_type : 17
price : 3704
price_currency : 5


In [6]:
print(zero_unique_collumns)
df = df.drop(labels=zero_unique_collumns, axis=1)

['type', 'furnished']


### Разделяю адрес на несколько стобцов для лучшей работы классификатора

In [7]:
def splitAddressCol(val: str):
    return val.split('/')

df = helpers.split_collumn(df, 'address', ['address1', 'address2', 'addres2'], splitAddressCol)

100%|██████████| 403487/403487 [00:00<00:00, 608567.51it/s]


### Разделю столбец с количеством комнат для лучшей работы классификатора

In [8]:
def splitRoomCol(val: str):
    result = re.findall(r"(\d+)\+(\d+)", val)
    if(len(result) == 1):
        return [int(result[0][1]), int(result[0][0]) + int(result[0][1])]
    else:
        return []

df = helpers.split_collumn(df, 'room_count', ['livingroom_count', 'room_count'], splitRoomCol)

100%|██████████| 403487/403487 [00:01<00:00, 355809.52it/s]


### Перевод цены в одну валюту

In [9]:
df = df[df['price_currency'].notna()]

In [10]:
def currencyConvert(row):
    return cc.convert(row['price'], row['price_currency'])

helpers.concat_collumn(df, ['price', 'price_currency'], 'price', currencyConvert)

100%|██████████| 402772/402772 [03:00<00:00, 2231.22it/s]


,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,size,heating_type,address1,address2,addres2,livingroom_count,room_count,price
id,,,,,,,,,,,,,,,,
1,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,90.0,Fancoil,İstanbul,Kartal,Kordonboyu,1.0,3.0,4.553793e+02
2,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,43.0,Fancoil,İstanbul,Kartal,Kordonboyu,0.0,1.0,6.375311e+04
3,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,NaN,Fancoil,Tekirdağ,Çorlu,Reşadiye,1.0,3.0,2.016680e+04
4,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,450.0,Fancoil,İstanbul,Beşiktaş,Levent,1.0,7.0,4.228522e+06
5,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,90.0,Fancoil,İstanbul,Kartal,Kordonboyu,1.0,3.0,1.886572e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403483,Daire,9/18/18,NaN,2,162,NaN,NaN,NaN,NaN,NaN,İstanbul,Sultanbeyli,Adil,NaN,NaN,1.951626e+02
403484,Daire,10/11/18,NaN,1,139,NaN,NaN,NaN,NaN,NaN,Sakarya,Adapazarı,Cumhuriyet,1.0,3.0,1.561301e+04
403485,Daire,11/22/18,NaN,1,97,NaN,NaN,NaN,NaN,NaN,Antalya,Alanya,Saray,1.0,2.0,4.800000e+04


## Сохранение отфильтрованных данных

In [11]:
df.to_csv('filtred_data.csv')

In [12]:
    df

,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,size,heating_type,price,price_currency,address1,address2,addres2,livingroom_count,room_count
id,,,,,,,,,,,,,,,,,
1,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,90.0,Fancoil,3500.0,TRY,İstanbul,Kartal,Kordonboyu,1.0,3.0
2,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,43.0,Fancoil,490000.0,TRY,İstanbul,Kartal,Kordonboyu,0.0,1.0
3,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,NaN,Fancoil,155000.0,TRY,Tekirdağ,Çorlu,Reşadiye,1.0,3.0
4,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,450.0,Fancoil,32500000.0,TRY,İstanbul,Beşiktaş,Levent,1.0,7.0
5,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,90.0,Fancoil,1450000.0,TRY,İstanbul,Kartal,Kordonboyu,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403483,Daire,9/18/18,NaN,2,162,NaN,NaN,NaN,NaN,NaN,1500.0,TRY,İstanbul,Sultanbeyli,Adil,NaN,NaN
403484,Daire,10/11/18,NaN,1,139,NaN,NaN,NaN,NaN,NaN,120000.0,TRY,Sakarya,Adapazarı,Cumhuriyet,1.0,3.0
403485,Daire,11/22/18,NaN,1,97,NaN,NaN,NaN,NaN,NaN,48000.0,EUR,Antalya,Alanya,Saray,1.0,2.0
